In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
from sage.graphs.trees import TreeIterator
import pandas as pd
from sage.graphs.connectivity import *

In [2]:
def pseudoLabGenerator(m,n):
    M = matrix(n*m)
    for i in range(n*m):
        for j in range(i,n*m):
            if (j+1)%n != 0 and j < n*m-1:
                M[j,j+1]=random.randint(0,1)
            if  j+n <n*m:
                M[j,j+n]=random.randint(0,1)
            M[j,i] = M[i,j]
    return Graph(M)

In [3]:
def allLabGenerator(n,m):
    L = []
    l=[]
    lim = binomial(2*m*n-n-m,(m-1)*(n-1))
    while len(L) != lim:
        G = pseudoLabGenerator(n,m)
        if G not in L and numberOfWay(G.adjacency_matrix())[1]==n*m-1:
            L.append(G)
    for i in L:
        if isPerfect(i):
            l.append(i)
    return l

In [5]:
def labGenerator(n,m):
    G = graphs.Grid2dGraph(n, m)
    pos = G.get_pos()
    dictio = G.random_spanning_tree(True).to_dictionary()
    for i in range(n):
        for j in range(m):
            dictio[j*n + i + 1] = dictio[(i, j)]
            for k in dictio:
                if ((i, j)) in dictio[k]:
                    dictio[k].append(j*n + i + 1)
                    dictio[k].pop(dictio[k].index((i, j)))
            dictio.pop((i,j))
    return Graph(dictio)

In [6]:
def labGenerator3D(n,m,l):
    return 0
    #sorry

In [7]:
def pseudoLabGenerator3D(n,m,l):
    M = matrix(n*m*l)
    for i in range(0,n*m*l):
        for j in range(i,n*m*l):
                if j%n != n-1:
                    M[j,j+1]=random.randint(0,1)
                if j//m != (1 + j//(n*m))*m-1:
                    M[j,j+m]=random.randint(0,1)
                if j+n*m < n*m*l:
                    M[j,j+n*m]=random.randint(0,1)
                M[j,i] = M[i,j]
    return Graph(M)

In [5]:
def drawLab(n, g):
    if (len(g.adjacency_matrix()[0])%n == 0):
        m = len(g.adjacency_matrix()[0])/n
        draw = (line2d([(  0,  0),
                        (  0,  n),
                        (  m,  n),
                        (  m,  0),
                        (  0,  0)], figsize=[m, n]))
        adj = g.adjacency_matrix()
        length = len(adj[0])-1
        for j in range(length):
            for h in range(j+1):
                u_x = j-h
                u_y = length-h
                if (u_y - u_x == 1) and ((u_x)%m < m-1):
                     if adj[u_x][u_y] == 0:
                        draw = draw + (line2d([(u_x % m +1, int(u_x/m)),
                                               (u_x % m +1, int(u_x/m)+1)], figsize=[m, n], color='red'))
                if (u_y - u_x == m):
                     if adj[u_x][u_y] == 0:
                        draw = draw + (line2d([(u_x % m   , int(u_x/m)+1),
                                               (u_x % m +1, int(u_x/m)+1)], figsize=[m, n], color = 'red'))
        memory = 0
        if 0 not in g.to_dictionary():
            for a in range (1, length+2):
                real = a + memory
                while real not in (g.to_dictionary()):
                    real = real + 1
                    memory = memory + 1
                draw = draw + text(real, ((a-1)%m+0.5, int(((a-1)/m))+0.5))
        else:
             for a in range (0, length+1):
                real = a + memory
                while real not in (g.to_dictionary()):
                    real = real + 1
                    memory = memory + 1
                draw = draw + text(real, (a%m+0.5, int(a/m)+0.5))
        return draw.show(axes=False)
    else:
        return ("La dimension demandee n'est pas correcte")

In [26]:
def dimensions(g):
    cells_counter = len(g.adjacency_matrix()[0])
    g = deleteHoles(g)
    references = g.to_dictionary()
    dimensions = []
    for i in (references):
        if (i != 0):
            if (cells_counter % i == 0):
                dimensions.append((i, cells_counter/i))
    return dimensions

In [6]:
def deleteHoles(g):
    dictio = g.to_dictionary()
    dictio2 = {}
    i = 1
    for number in dictio:
        dictio2[i] = dictio[number]
        for k in dictio:
            if number in dictio[k]:
                dictio[k].append(i)
                dictio[k].pop(dictio[k].index(number))
        i += 1  
    return Graph(dictio2)

In [3]:
def areDimensionsAlterable(g):
    g = deleteHoles(g)
    dictio = g.to_dictionary()
    for(n, m) in dimensions(g):
        if (n != 1 and m !=1):
            end_list = []
            find = []
            for i in range(1, len(g.adjacency_matrix()[0]) - n + 1):
                if (i+n) not in dictio[i]:
                    if (i, i+n) not in find:
                         find.append((i, i+n))
            for i in range(1, n*m):
                if i%n == 0:
                    end_list.append(i)
            compteur = 0
            count_walls = len(find)
            for end_line in (end_list):
                if (end_line + 1) not in dictio[end_line]:
                    if len(find) > 0:
                        find.pop(0)
                    else:
                        return False
            if (len(find) > 0):
                return False
    return true

In [12]:
def numberOfWay(M):
    n1= 0
    for i in range(len(M[0])):
        for j in range(i+1, len(M[0])):
            if (M[i][j]==1):
                n1=n1 + 1
    return ([1, n1])

In [13]:
def isPerfect(G):
    M=G.adjacency_matrix()
    for i in range(M.nrows()):
        for j in range(i, M.ncols()):
            if not isWay(i, j, G):
                return false
    for k in range(M.nrows()):
        for l in range(k+1, M.ncols()):
            if (M[k, l]==1):
                M[k, l]=0
                M[l, k]=0
                i=0
                H=Graph(M)
                for i in range(M.nrows()):
                    for j in range(i, M.ncols()):
                        if not isWay(i, j, H):
                            i=i+1
                    if i==0:
                        return false
                M[k, l]=1
                M[l, k]=1
    return true

In [14]:
def isWay(c1, c2, G):
    M=G.adjacency_matrix()
    seen = []
    tosee = [c1]
    while tosee != []:
        for s in tosee:
            for i in range(M.nrows()):
                if (M[i][s]==1):
                    if i not in seen:
                        tosee.append(i)
            seen.append(s)
            tosee.remove(s)
    if c2 in seen:
        return true
    else:
        return false

In [15]:
def howManyPerfect(n, m, nbe):
    np=0
    for i in range(nbe):
        G=pseudoLabGenerator(n, m)
        if(isPerfect(G)):
            np=np+1
    return (np*100.)/nbe

In [16]:
def averageGenerator(n, m, nbe):
    l=[]
    g=[]
    np=0
    for i in range(nbe):
        G=pseudoLabGenerator(n, m)
        if(isPerfect(G)):
            np=np+1
        l.append(numberOfWay(G.adjacency_matrix()))
    for i in range(nbe-1):
        for j in range(i+1, nbe):
            if l[i][1]==l[j][1]:
                if j not in g:
                    g.append(j)
                l[i][0]=l[i][0]+1

    g.sort(reverse=true)
    for i in range(len(g)):
        l.pop(g[i])
    l.sort(reverse=true)
    for i in range(len(l)):
        l[i][0]=float(l[i][0]*100./nbe)
    return ((np*100.)/nbe), l

In [17]:
def modelisationSimul(n, m, repetitions=100):
    result = averageGenerator(n, m, repetitions)
    plot = result[1]
    data = {}
    errorValues = {}
    #On initialise tous les pourcentages à 0% parce que si l'algorithme d'averageGenerator ne rencontre jamais x murs (parce que la proba est très faible) cela peut poser des problèmes au niveau des indices
    for i in range (m*(2*n-1)-n+1): #Formule du nombre maximal de murs dans un PL, c'est l'occasion de la vérifier
        data[i] = 0
        errorValues[i] = int(0)
    for i in range (len(plot)):
        data[plot[i][1]] = plot[i][0]

    true_data = {}
    for i in range (m*(2*n-1)-n+1):
        true_data[i] = float(data[i])

    names = list(true_data.keys())
    values = list(true_data.values())
    for i in true_data:
        errorValues[i] = true_data[i] - (true_data[i]+true_data[len(true_data)-1-i])/2
    plt.title('% de chances d avoir x chemins dans un PL de taille {} * {}'.format(n, m))

    for i in range (m*(2*n-1)-n, -1, -1):
        if values[i] == 0:
            values.pop(i)
            names.pop(i)
            errorValues.pop(i)
    plt.bar(names, values)
    plt.text(names[0], -5, "Dont {}% de parfaits".format(round(result[0], 3)))
    plt.text(names[0], -8, "Soit {}% des PL ayant le bon nombre de murs".format(round(result[0]*100/float(true_data[n*m-1]), 3)))
    for i in names:
        plt.errorbar(i, values[names.index(i)], yerr = float(errorValues[i]), fmt = 'none', ecolor = 'red', elinewidth = errorValues[i])

    plt.show()

In [18]:
def tableNbLab(m):
    n = m+1
    t = np.zeros((n,n),dtype=int)
    for i in range(1, n):
        for j in range(i,n):
            t[i][j] = len(allLabGenerator(i,j));
            t[j][i] = t[i][j]
    return pd.DataFrame(t)

In [2]:
def compteMaxCC(listGraphs) :                                                     #Compte le nombre maximum de composantes connexes dans la liste de graphes listGraphs
    nbMaxCC = 1
    graphMaxCC = []
    for i in range (len(listGraphs)) :
        nbCC = connected_components_number(listGraphs[i])
        if nbCC > nbMaxCC :
            nbMaxCC = nbCC
            graphMaxCC = listGraphs[i].copy()
    return [nbMaxCC, graphMaxCC, connected_components(graphMaxCC)]                 #Extrait le nombre maximum de composantes connexes, le graphe les contenant, et les composantes connexes en question